
# **PROJETO APLICADO IV — Entrega 3 (Notebook consolidado e reprodutível)**
**Tema:** Predição da Taxa de Desemprego no Brasil (PNAD) com CAGED e SELIC  
**Modelos:** SARIMAX (estatístico) e LGBM Regressor (ML)

> Execute as células na ordem. Para reproduzir métricas de experimentos anteriores,
> ajuste `CUT_END` na seção de Configurações.


In [ ]:

# === Dependências (instala se necessário) ===
import importlib, sys, subprocess

def ensure(pkg):
    try:
        importlib.import_module(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])

for pkg in ["lightgbm", "statsmodels", "pandas", "numpy", "matplotlib", "scipy"]:
    ensure(pkg)
print("Ok: dependências disponíveis.")


In [ ]:

import warnings, os, math, json, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm

warnings.filterwarnings("ignore")
np.random.seed(42); random.seed(42)

plt.rcParams["figure.figsize"] = (7,5)
plt.rcParams["axes.grid"] = True



## **Configurações**
- `USE_LOCAL`: usa CSV local (`LOCAL_PATH`) ou baixa do repositório (`RAW_URL`).
- `CUT_END`: data que congela o fim do treino (inclusive). Se `None`, usa holdout dos últimos `TEST_LAST_N` trimestres.


In [ ]:

import pandas as pd

USE_LOCAL = False
LOCAL_PATH = "dataset/tratados/caged_pnad_selic_trimestral.csv"
RAW_URL = (
    "https://raw.githubusercontent.com/"
    "fpaterni10/projeto-aplicado-iv-desemprego-br/main/"
    "dataset/tratados/caged_pnad_selic_trimestral.csv"
)

CUT_END = None   # ex.: pd.Timestamp("2024-06-30")
TEST_LAST_N = 4
SEED = 42



## **Setup & Carregamento**
Cria `df_model` com índice trimestral e *features* derivadas.


In [ ]:

def load_base():
    import numpy as np
    import pandas as pd
    import os
    if USE_LOCAL and os.path.exists(LOCAL_PATH):
        df0 = pd.read_csv(LOCAL_PATH)
    else:
        df0 = pd.read_csv(RAW_URL)
    df0["periodo_tri"] = df0["periodo_tri"].astype(str)
    per = pd.PeriodIndex(df0["periodo_tri"], freq="Q-DEC")
    df = df0.set_index(per).sort_index()
    df.index.name = "periodo"
    df["data"] = df.index.to_timestamp(how="end").normalize()

    df["caged_roll3"] = df["caged_saldo_tri"].rolling(3, min_periods=3).mean()
    df["caged_roll3_asinh"] = np.arcsinh(df["caged_roll3"])
    df["quarter"] = df.index.quarter
    Q = pd.get_dummies(df["quarter"], prefix="Q", drop_first=False)
    df["post_2021"] = (df["data"] >= "2021-01-01").astype(int)

    dfm = pd.concat([df, Q], axis=1).dropna().copy()

    for k in [1,2,3,4]:
        dfm[f"tx_lag{k}"] = dfm["tx_desocupacao"].shift(k)
    dfm["caged_diff1"] = dfm["caged_saldo_tri"].diff(1)
    dfm["selic_diff1"] = dfm["selic_tri"].diff(1)
    dfm["caged_roll3_std"] = dfm["caged_saldo_tri"].rolling(3).std()
    dfm = dfm.dropna().copy()
    return df, dfm

df, df_model = load_base()
print("Dimensão df_model:", df_model.shape)
df_model.info()



## **Utilitários**
Funções de split temporal, métricas, gráficos e validação.


In [ ]:

import numpy as np, math, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm

def temporal_holdout(df, n_test=4, cut_end=None):
    dfx = df.sort_values("data").copy()
    if cut_end is not None:
        tr = dfx[dfx["data"] <= cut_end].copy()
        te = dfx[dfx["data"] > cut_end].head(n_test).copy()
        return tr, te
    return dfx.iloc[:-n_test].copy(), dfx.iloc[-n_test:].copy()

def metrics_report(y_true, y_pred, prefix="modelo"):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    n = min(len(y_true), len(y_pred))
    y_true = y_true[:n]; y_pred = y_pred[:n]
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    denom = np.where(np.abs(y_true) < 1e-12, np.nan, y_true)
    mape = np.nanmean(np.abs((y_true - y_pred) / denom)) * 100
    return pd.DataFrame({"MAE":[mae], "RMSE":[rmse], "MAPE (%)":[mape]}, index=[prefix])

def plot_real_vs_pred(t, y_true, preds: dict, title="Real vs Pred — Holdout"):
    fig, ax = plt.subplots()
    ax.plot(t, y_true, label="Real")
    for name, yhat in preds.items():
        ax.plot(t, yhat, label=name)
    ax.set_title(title); ax.legend(); fig.tight_layout()
    return fig, ax

def diagnosticos_residuos(resid, lags=12, title_prefix=""):
    resid = pd.Series(resid).dropna()
    fig1 = plt.figure(figsize=(7,3.5))
    plt.plot(resid.values); plt.title(f"{title_prefix} Resíduos")
    fig2 = sm.graphics.tsa.plot_acf(resid, lags=lags)
    plt.title(f"{title_prefix} ACF dos Resíduos")
    lb = sm.stats.acorr_ljungbox(resid, lags=[4,8,12], return_df=True)
    jb = sm.stats.jarque_bera(resid)
    return fig1, fig2, lb, jb

def timeseries_cv_metrics(X, y, model_factory, n_splits=4):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    rows = []
    for fold, (tr_idx, va_idx) in enumerate(tscv.split(X), start=1):
        Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
        ytr, yva = y.iloc[tr_idx], y.iloc[va_idx]
        model = model_factory()
        model.fit(Xtr, ytr)
        p = model.predict(Xva)
        rows.append({"fold": fold,
                     "MAE": mean_absolute_error(yva, p),
                     "RMSE": math.sqrt(mean_squared_error(yva, p))})
    dfm = pd.DataFrame(rows)
    avg = dfm[["MAE","RMSE"]].mean().to_frame().T
    avg["fold"] = "média"
    return pd.concat([dfm, avg], ignore_index=True)



## **Auditoria rápida dos dados**
Checagens de frequência temporal, monotonicidade, duplicidades e nulos.


In [ ]:

ok_freq = (df_model.index.freqstr == "Q-DEC") or (getattr(df_model.index, "freqstr", None) == "Q-DEC")
monot = df_model["data"].is_monotonic_increasing
dups = df_model.index.duplicated().sum()
nulls = df_model.isna().sum().sort_values(ascending=False).head(10)
print({"freq_QDEC": ok_freq, "monotonic_data": monot, "duplicated_index": int(dups)})
print("
Top-10 colunas com nulos:
", nulls[nulls>0])



## **Modelo 1 — LGBM (calibrado)**
Features enxutas (CAGED + sazonais + regime) e **calibração de viés**.


In [ ]:

from lightgbm import LGBMRegressor

features_cal = ['caged_roll3_asinh','caged_saldo_tri','Q_1','Q_2','Q_3','Q_4','post_2021']
features_cal = [f for f in features_cal if f in df_model.columns]
target = 'tx_desocupacao'

train_lgb, test_lgb = temporal_holdout(df_model, n_test=TEST_LAST_N, cut_end=CUT_END)
X_tr, y_tr = train_lgb[features_cal], train_lgb[target].to_numpy()
X_te, y_te = test_lgb[features_cal], test_lgb[target].to_numpy()

lgbm = LGBMRegressor(
    n_estimators=400, learning_rate=0.05,
    num_leaves=31, max_depth=4,
    min_child_samples=20, subsample=0.8, colsample_bytree=0.8,
    reg_lambda=0.1, random_state=SEED
)
lgbm.fit(X_tr, y_tr)
pred_tr = lgbm.predict(X_tr)
bias_k = 4
bias = (y_tr[-bias_k:] - pred_tr[-bias_k:]).mean()
pred_lgb = lgbm.predict(X_te) + bias

lgbm_metrics = metrics_report(y_te, pred_lgb, prefix="LGBM (calibrado)")
display(lgbm_metrics)

t_holdout = test_lgb["data"]
fig, ax = plot_real_vs_pred(t_holdout, y_te, {"LGBM (calibrado)": pred_lgb},
                            title="Real vs Pred — LGBM (calibrado)")
os.makedirs("docs/figuras", exist_ok=True)
fig.savefig("docs/figuras/real_vs_pred_lgbm_calibrado.png", dpi=160, bbox_inches="tight")
plt.show(); plt.close(fig)



## **Modelo 2 — SARIMAX (baseline estatístico)**
Baseline com exógena do **CAGED** (`caged_saldo_tri`). Ordem `(1,1,2) × (1,0,1,4)`.


In [ ]:

exog_cols_base = ['caged_saldo_tri']
exog_cols_base = [c for c in exog_cols_base if c in df_model.columns]

train_sar, test_sar = temporal_holdout(df_model, n_test=TEST_LAST_N, cut_end=CUT_END)

y_tr = train_sar['tx_desocupacao'].asfreq('Q-DEC')
y_te = test_sar['tx_desocupacao'].asfreq('Q-DEC')
X_tr = train_sar[exog_cols_base].asfreq('Q-DEC')
X_te = test_sar[exog_cols_base].asfreq('Q-DEC')

sarimax = SARIMAX(
    endog=y_tr, exog=X_tr,
    order=(1,1,2), seasonal_order=(1,0,1,4),
    enforce_stationarity=False, enforce_invertibility=False
).fit(disp=False, maxiter=500)

pred_sar = sarimax.get_forecast(steps=len(y_te), exog=X_te).predicted_mean
sarimax_metrics = metrics_report(y_te.values, pred_sar.values, prefix="SARIMAX baseline")
display(sarimax_metrics)

fig_r, fig_acf, lb, jb = diagnosticos_residuos(sarimax.resid, lags=12, title_prefix="SARIMAX —")
print("\nLjung-Box (p-valores):\n", lb)
print("\nJarque-Bera:", jb)

os.makedirs("docs/figuras", exist_ok=True)
fig_r.savefig("docs/figuras/sarimax_residuos.png", dpi=160, bbox_inches="tight")
fig_acf.savefig("docs/figuras/sarimax_residuos_acf.png", dpi=160, bbox_inches="tight")
plt.show(); plt.close(fig_r); plt.close(fig_acf)

fig, ax = plot_real_vs_pred(t_holdout, y_te.values, {"SARIMAX": pred_sar.values},
                            title="Real vs Pred — SARIMAX (baseline)")
fig.savefig("docs/figuras/sarimax_real_vs_pred.png", dpi=160, bbox_inches="tight")
plt.show(); plt.close(fig)



### **Experimento de ablação: SARIMAX com SELIC**
Compara baseline (CAGED) vs CAGED+SELIC.


In [ ]:

exog_cols_selic = ['caged_saldo_tri', 'caged_roll3_asinh', 'selic_tri']
exog_cols_selic = [c for c in exog_cols_selic if c in df_model.columns]

X_tr2 = train_sar[exog_cols_selic].asfreq('Q-DEC')
X_te2 = test_sar[exog_cols_selic].asfreq('Q-DEC')

sarimax_selic = SARIMAX(
    endog=y_tr, exog=X_tr2,
    order=(1,1,2), seasonal_order=(1,0,1,4),
    enforce_stationarity=False, enforce_invertibility=False
).fit(disp=False, maxiter=500)

pred_sar2 = sarimax_selic.get_forecast(steps=len(y_te), exog=X_te2).predicted_mean
sarimax_selic_metrics = metrics_report(y_te.values, pred_sar2.values, prefix="SARIMAX (+SELIC)")
display(sarimax_selic_metrics)



## **Validação temporal (TimeSeriesSplit) — LGBM**
Relatório de métricas por dobra e média.


In [ ]:

from lightgbm import LGBMRegressor

X = df_model[features_cal].copy()
y = df_model[target].copy()

def factory():
    return LGBMRegressor(
        n_estimators=400, learning_rate=0.05,
        num_leaves=31, max_depth=4,
        min_child_samples=20, subsample=0.8,
        colsample_bytree=0.8, reg_lambda=0.1, random_state=SEED
    )

cv_report = timeseries_cv_metrics(X, y, factory, n_splits=4)
display(cv_report)



## **Comparação entre Modelos (holdout)**
Geração dinâmica (sem números fixos).


In [ ]:

comparativo = pd.concat([lgbm_metrics, sarimax_metrics, sarimax_selic_metrics]).round(4)
display(comparativo)

try:
    gain_mae = 100*(sarimax_metrics["MAE"].iloc[0] - lgbm_metrics["MAE"].iloc[0]) / sarimax_metrics["MAE"].iloc[0]
    gain_rmse = 100*(sarimax_metrics["RMSE"].iloc[0] - lgbm_metrics["RMSE"].iloc[0]) / sarimax_metrics["RMSE"].iloc[0]
    print(f"Ganho do LGBM vs SARIMAX baseline — MAE: {gain_mae:.1f}% | RMSE: {gain_rmse:.1f}%")
except Exception as e:
    print("Observação:", e)



## **Conclusão parcial**
- Modelos implementados (SARIMAX e LGBM) e avaliados em holdout.
- Validação temporal (TimeSeriesSplit) reportada para o LGBM.
- Diagnóstico formal dos resíduos do SARIMAX (ACF + Ljung‑Box + Jarque‑Bera).
- Ablação da SELIC no SARIMAX.
- Tabela comparativa **gerada por código**, atendendo ao requisito de reprodutibilidade.
